In [1]:
!ls data/

SMSSpamCollection.txt  out  simple_text.txt


In [37]:
! wget https://hrcdn.net/s3_pub/istreet-assets/H4_TQkbOj39HUNoBukluIQ/training.txt

--2021-02-28 23:59:49--  https://hrcdn.net/s3_pub/istreet-assets/H4_TQkbOj39HUNoBukluIQ/training.txt
Resolving hrcdn.net (hrcdn.net)... 23.212.254.99, 23.212.254.56, 2600:140f:4::6011:960b, ...
Connecting to hrcdn.net (hrcdn.net)|23.212.254.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: 'training.txt'

training.txt            [   <=>              ] 444.49K   708KB/s    in 0.6s    

2021-02-28 23:59:50 (708 KB/s) - 'training.txt' saved [455154]



In [38]:
messages = pd.read_csv('training.txt', 
                       sep='\t', 
                       quoting=csv.QUOTE_NONE,
                       names=["label", "message"])

In [41]:
messages.groupby('label').describe()

message                                  
        count unique                   top freq
label                                          
0        3091    639  I hate Harry Potter.   86
1        3995    772  I love Harry Potter.  167

In [4]:
import pandas as pd
import csv
#Data Loading
messages = [line.rstrip() for line in open('data/SMSSpamCollection.txt')]
print(len(messages))
#Appending column headers
messages = pd.read_csv('data/SMSSpamCollection.txt', 
                       sep='\t', 
                       quoting=csv.QUOTE_NONE,
                       names=["label", "message"])

5574


In [5]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will ü b going to esplanade fr home?
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


In [6]:
data_size=messages.shape
print(data_size)

(5574, 2)


In [7]:
messages_col_names=list(messages.columns)
print(messages_col_names)

['label', 'message']


In [8]:
print(messages.groupby('label').describe())

      message                                                               
        count unique                                                top freq
label                                                                       
ham      4827   4518                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4


In [9]:
#Identifying the outcome/target variable.
message_target=messages['label'] 
print(message_target)

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5569    spam
5570     ham
5571     ham
5572     ham
5573     ham
Name: label, Length: 5574, dtype: object


In [14]:
import sys
if sys.version_info[0] >= 3:
    unicode = str


In [16]:
import nltk
# nltk.download('all')
from nltk.tokenize import word_tokenize
def split_tokens(message):
  message=message.lower()
#   message = unicode(message, 'utf8') #convert bytes into proper unicode
  word_tokens =word_tokenize(message)
  return word_tokens
messages['tokenized_message'] = messages.apply(lambda row: split_tokens(row['message']),axis=1)

In [42]:
split_tokens("if you cannot do great things, do small things in a great way")

['if',
 'you',
 'can',
 'not',
 'do',
 'great',
 'things',
 ',',
 'do',
 'small',
 'things',
 'in',
 'a',
 'great',
 'way']

In [17]:
from nltk.stem.wordnet import WordNetLemmatizer
def split_into_lemmas(message):
    lemma = []
    lemmatizer = WordNetLemmatizer()
    for word in message:
        a=lemmatizer.lemmatize(word)
        lemma.append(a)
    return lemma
messages['lemmatized_message'] = messages.apply(lambda row: split_into_lemmas(row['tokenized_message']),axis=1)
print('Tokenized message:',messages['tokenized_message'][11])
print('Lemmatized message:',messages['lemmatized_message'][11])

Tokenized message: ['six', 'chances', 'to', 'win', 'cash', '!', 'from', '100', 'to', '20,000', 'pounds', 'txt', '>', 'csh11', 'and', 'send', 'to', '87575.', 'cost', '150p/day', ',', '6days', ',', '16+', 'tsandcs', 'apply', 'reply', 'hl', '4', 'info']
Lemmatized message: ['six', 'chance', 'to', 'win', 'cash', '!', 'from', '100', 'to', '20,000', 'pound', 'txt', '>', 'csh11', 'and', 'send', 'to', '87575.', 'cost', '150p/day', ',', '6days', ',', '16+', 'tsandcs', 'apply', 'reply', 'hl', '4', 'info']


In [18]:
from nltk.corpus import stopwords
def stopword_removal(message):
    stop_words = set(stopwords.words('english'))
    filtered_sentence = []
    filtered_sentence = ' '.join([word for word in message if word not in stop_words])
    return filtered_sentence
messages['preprocessed_message'] = messages.apply(lambda row: stopword_removal(row['lemmatized_message']),axis=1)
Training_data=pd.Series(list(messages['preprocessed_message']))
Training_label=pd.Series(list(messages['label']))

In [19]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),
                                   min_df = (1/len(Training_label)), 
                                   max_df = 0.7)
Total_Dictionary_TFIDF = tfidf_vectorizer.fit(Training_data)
message_data_TFIDF = Total_Dictionary_TFIDF.transform(Training_data)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
tf_vectorizer = CountVectorizer(ngram_range=(1, 2),min_df = (1/len(Training_label)), max_df = 0.7)
Total_Dictionary_TDM = tf_vectorizer.fit(Training_data)
message_data_TDM = Total_Dictionary_TDM.transform(Training_data)

In [22]:
from sklearn.model_selection import train_test_split#Splitting the data for training and testing
train_data,test_data, train_label, test_label = train_test_split(message_data_TDM, Training_label, test_size=.1)

In [34]:
test_label

3955     ham
5279     ham
4241     ham
47       ham
312     spam
        ... 
4559     ham
3679    spam
455     spam
4109     ham
5342     ham
Length: 558, dtype: object

In [23]:
from sklearn.tree import DecisionTreeClassifier#Creating a decision classifier model
classifier=DecisionTreeClassifier() #Model training
classifier = classifier.fit(train_data, train_label) #After being fitted, the model can then be used to predict the output.
message_predicted_target = classifier.predict(test_data)
score = classifier.score(test_data, test_label)
print('Decision Tree Classifier : ',score)

Decision Tree Classifier :  0.978494623655914


In [24]:
seed=7
from sklearn.linear_model import SGDClassifier
classifier =  SGDClassifier(loss='modified_huber', shuffle=True,random_state=seed)
classifier = classifier.fit(train_data, train_label)
score = classifier.score(test_data, test_label)
print('SGD classifier : ',score)

SGD classifier :  0.989247311827957


In [25]:
from sklearn.svm import SVC
classifier = SVC(kernel="linear", C=0.025,random_state=seed)
classifier = classifier.fit(train_data, train_label)
score = classifier.score(test_data, test_label)
print('SVM Classifier : ',score)

SVM Classifier :  0.9838709677419355


In [26]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=10,random_state=seed)
classifier = classifier.fit(train_data, train_label)
score = classifier.score(test_data, test_label)
print('Random Forest Classifier : ',score)

Random Forest Classifier :  0.8763440860215054


In [27]:
# Tuning
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=5, n_estimators=15, max_features=60,random_state=seed)
classifier = classifier.fit(train_data, train_label)
score=classifier.score(test_data, test_label)
print('Random Forest classification after model tuning',score)

Random Forest classification after model tuning 0.8763440860215054


In [28]:
seed=7
from sklearn.model_selection import StratifiedShuffleSplit
###cross validation with 10% sample size
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.1, random_state=seed)
sss.get_n_splits(message_data_TDM,Training_label)
print(sss)

StratifiedShuffleSplit(n_splits=1, random_state=7, test_size=0.1,
            train_size=None)


In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
classifiers = [
    DecisionTreeClassifier(),
    SGDClassifier(loss='modified_huber', shuffle=True),
    SVC(kernel="linear", C=0.025),
    KNeighborsClassifier(),
    OneVsRestClassifier(svm.LinearSVC()),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=10),
   ]
for clf in classifiers:
    score=0
    for train_index, test_index in sss.split(message_data_TDM,Training_label):
        X_train, X_test = message_data_TDM [train_index], message_data_TDM [test_index]
        y_train, y_test = Training_label[train_index], Training_label[test_index]
        clf.fit(X_train, y_train)
        score=score+clf.score(X_test, y_test)
    print(score)

0.9695340501792115
0.96415770609319
0.9695340501792115
0.9014336917562724
0.974910394265233
0.8655913978494624


In [31]:
from sklearn.metrics import accuracy_score
print('Accuracy Score',accuracy_score(test_label,message_predicted_target))  
classifier = classifier.fit(train_data, train_label)
score=classifier.score(test_data, test_label)
test_label.value_counts()

Accuracy Score 0.978494623655914


ham     489
spam     69
dtype: int64

In [33]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix\n',confusion_matrix(test_label,message_predicted_target))

Confusion Matrix
 [[486   3]
 [  9  60]]


In [35]:
from sklearn.metrics import classification_report
target_names = ['spam', 'ham']
print(classification_report(test_label, message_predicted_target, target_names=target_names))

              precision    recall  f1-score   support

        spam       0.98      0.99      0.99       489
         ham       0.95      0.87      0.91        69

    accuracy                           0.98       558
   macro avg       0.97      0.93      0.95       558
weighted avg       0.98      0.98      0.98       558

